In [2]:
from gurobipy import *
import matplotlib.pyplot as plt
from gurobipy import GRB

def modelLPConvBounded (times,origin,goal,vel,mdl):

    xO, yO = origin
    xG, yG = goal
    vxO,vyO = vel
    # Parameters equação de dinâmica
    deltaT = 1.0
    m = 2.0

    nWP = len(times)-1
    M = 1000.0
    #print(nWP)

    # Variáveis do modelo
    x = mdl.addVars(times, name='x')
    y = mdl.addVars(times, name='y')
    z = mdl.addVars(times, 4, vtype=GRB.BINARY, name='z')
    z2 = mdl.addVars(times, 4, vtype=GRB.BINARY, name='z')
    vx = mdl.addVars(times, name='vx', lb=-1, ub=1)
    vy = mdl.addVars(times, name='vy', lb=-1, ub=1)
    Fx = mdl.addVars(times,name='Fx', lb=-1, ub=1)
    Fy = mdl.addVars(times,name='Fy', lb=-1, ub=1)

    #obj = mdl.addVar(name='obj')

    # Restrições de dinâmica do veículo
    mdl.addConstrs((x[times[t+1]]==x[times[t]]+vx[times[t]]*deltaT+0.5*(deltaT**2)*Fx[times[t]]/m for t in range(len(times)) if times[t] != times[nWP]), name ='xpos')
    mdl.addConstrs((y[times[t+1]]==y[times[t]]+vy[times[t]]*deltaT+0.5*(deltaT**2)*Fy[times[t]]/m for t in range(len(times)) if times[t] != times[nWP]), name ='ypos')
    mdl.addConstrs((vx[times[t+1]]==vx[times[t]]+deltaT*Fx[times[t]]/m for t in range(len(times)) if times[t] != times[nWP]) , name='vx')
    mdl.addConstrs((vy[times[t+1]]==vy[times[t]]+deltaT*Fy[times[t]]/m for t in range(len(times))if times[t] != times[nWP]), name='vy')

    # Initial Position Constraints
    mdl.addConstrs((x[times[t]]==xO for t in range(len(times)) if times[t] == times[0]), name='x0')
    mdl.addConstrs((y[times[t]]==yO for t in range(len(times)) if times[t] == times[0]), name='y0')
    mdl.addConstrs((vx[times[t]]==vxO for t in range(len(times)) if times[t] == times[0]), name='x0')
    mdl.addConstrs((vy[times[t]]==vyO for t in range(len(times)) if times[t] == times[0]), name='y0')

    #Regiao Factível
    mdl.addConstrs((x[times[t]] - 9*y[times[t]] <= -8 for t in range(len(times))), name='face1')
    mdl.addConstrs((-9*x[times[t]] + y[times[t]] <= -8 for t in range(len(times))), name='face2')
    mdl.addConstrs((x[times[t]] + 7*y[times[t]] <= 72 for t in range(len(times))), name='face3')
    mdl.addConstrs((7*x[times[t]] + y[times[t]] <= 72 for t in range(len(times))), name='face4')


    #Obstáculo 1
    mdl.addConstrs((3 - y[times[t]] <= M*(1-z[times[t],0]) for t in range(len(times))), name='face1')
    mdl.addConstrs((3.5-x[times[t]] <= M*(1-z[times[t],1]) for t in range(len(times))), name='face2')
    mdl.addConstrs((y[times[t]] -2 <= M*(1-z[times[t],2]) for t in range(len(times))), name='face3')
    mdl.addConstrs((x[times[t]] -2 <= M*(1-z[times[t],3]) for t in range(len(times))), name='face4')

    mdl.addConstrs((1 - z[times[t],0] - z[times[t],1] - z[times[t],2] - z[times[t],3] <= 0)  for t in range(len(times)))

    #Obstáculo 2
    mdl.addConstrs((7 - y[times[t]] <= M*(1-z2[times[t],0]) for t in range(len(times))), name='face1')
    mdl.addConstrs((7-x[times[t]] <= M*(1-z2[times[t],1]) for t in range(len(times))), name='face2')
    mdl.addConstrs((y[times[t]] -3 <= M*(1-z2[times[t],2]) for t in range(len(times))), name='face3')
    mdl.addConstrs((x[times[t]] -5 <= M*(1-z2[times[t],3]) for t in range(len(times))), name='face4')

    mdl.addConstrs((1 - z2[times[t],0] - z2[times[t],1] - z2[times[t],2] - z2[times[t],3] <= 0)  for t in range(len(times)))

    #Objective function

    mdl.modelSense = GRB.MINIMIZE
    mdl.setObjective(((x[times[len(times)-1]] - xG)**2 +(y[times[len(times)-1]] -yG)**2) + vx[times[len(times)-1]]**2 + vy[times[len(times)-1]]**2)

    mdl.optimize()

    objectiveValue = mdl.objVal

    temp_X = []
    temp_Y = []
    temp_F = []
    temp_Vx = []
    temp_Vy = []
    temp_Fx = []
    temp_Fy = []

    for t in range(len(times)):
        temp_X.append(float(x[times[t]].x))
        temp_Y.append(float(y[times[t]].x))
        temp_F.append([float(Fx[times[t]].x),float(Fy[times[t]].x)])

    for t in range(len(times)):
        temp_Vx.append(float(vx[times[t]].x))
        temp_Vy.append(float(vy[times[t]].x))
        temp_Fx.append(float(Fx[times[t]].x))
        temp_Fy.append(float(Fy[times[t]].x))




    return objectiveValue



In [ ]:
def trajetoria():

  from matplotlib.path import Path
  import matplotlib.patches as patches
  from numpy import random

  verts = [
    (1., 1.),  # left, bottom
    (2., 10.),  # left, top
    (9., 9.),  # right, top
    (10., 2.),  # right, bottom
    (0., 0.),  # ignored
  ]

  codes = [
      Path.MOVETO,
      Path.LINETO,
      Path.LINETO,
      Path.LINETO,
      Path.CLOSEPOLY,
  ]

  path = Path(verts, codes)

  fig, ax = plt.subplots()
  patch = patches.PathPatch(path, facecolor='orange', lw=2)
  ax.add_patch(patch)

  verts2 = [
      (2., 2.),  # left, bottom
      (2., 3.),  # left, top
      (3.5, 3.),  # right, top
      (3.5, 2.),  # right, bottom
      (0., 0.),  # ignored
    ]

  codes2 = [
        Path.MOVETO,
        Path.LINETO,
        Path.LINETO,
        Path.LINETO,
        Path.CLOSEPOLY,
    ]

  path2 = Path(verts2, codes2)

  #fig, ax = plt.subplots()
  patch2 = patches.PathPatch(path2, facecolor='blue', lw=2)
  ax.add_patch(patch2)

  verts3 = [
      (5., 3.),  # left, bottom
      (5., 7.),  # left, top
      (7, 7.),  # right, top
      (7, 3.),  # right, bottom
      (0., 0.),  # ignored
    ]

  codes3 = [
        Path.MOVETO,
        Path.LINETO,
        Path.LINETO,
        Path.LINETO,
        Path.CLOSEPOLY,
      ]

  path3 = Path(verts3, codes3)

  #fig, ax = plt.subplots()
  patch3 = patches.PathPatch(path3, facecolor='blue', lw=2)
  ax.add_patch(patch3)


  #Ajusta o limite da área de plot
  ax.set_xlim(-1, 11)
  ax.set_ylim(-1, 11)

  temp_Xr = []
  temp_Yr = []
  for t in range(len(times)):
    temp_Xr.append(temp_X[t]+temp_Vx[t]*deltaT+0.5*(deltaT**2)*temp_Fx[t]/m + random.normal(0,desvio))
    temp_Yr.append(temp_Y[t]+temp_Vy[t]*deltaT+0.5*(deltaT**2)*temp_Fy[t]/m+ random.normal(0,desvio))